# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import pickle

from tqdm import tqdm
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

# Load Dataset

In [2]:
# # For Google Colab (mount google drive)
# from google.colab import drive

# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/Portfolio/DS Projects/Depression Risk/Student Depression Dataset.csv')

In [3]:
# For Local Environment
df = pd.read_csv('dataset/Student Depression Dataset.csv')

In [4]:
df.head()

,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  object 
 11  Di

In [6]:
df.describe()

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27901.000000,27898.000000,27901.000000
mean,70442.149421,25.822300,3.141214,0.000430,7.656104,2.943837,0.000681,7.156984,3.139867,0.585499
std,40641.175216,4.905687,1.381465,0.043992,1.470707,1.361148,0.044394,3.707642,1.437347,0.492645
min,2.000000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,35039.000000,21.000000,2.000000,0.000000,6.290000,2.000000,0.000000,4.000000,2.000000,0.000000
50%,70684.000000,25.000000,3.000000,0.000000,7.770000,3.000000,0.000000,8.000000,3.000000,1.000000
75%,105818.000000,30.000000,4.000000,0.000000,8.920000,4.000000,0.000000,10.000000,4.000000,1.000000
max,140699.000000,59.000000,5.000000,5.000000,10.000000,5.000000,4.000000,12.000000,5.000000,1.000000


# Data Preprocessing

In [7]:
missing_values = df.isnull().sum()
print(missing_values)

id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         3
Family History of Mental Illness         0
Depression                               0
dtype: int64


In [8]:
# Drop rows with missing values from the DataFrame df
df_cleaned = df.copy()
df_cleaned = df_cleaned.dropna()

# Optionally, you can print the shape of the original and cleaned DataFrames
print(f"Original shape: {df.shape}")
print(f"Cleaned shape: {df_cleaned.shape}")

# If you want to check the missing values again after dropping
missing_values_after = df_cleaned.isnull().sum()
print("Missing values after dropping rows:")
print(missing_values_after)

Original shape: (27901, 18)
Cleaned shape: (27898, 18)
Missing values after dropping rows:
id                                       0
Gender                                   0
Age                                      0
City                                     0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64


In [9]:
# Drop irrelevant feature (based on domain knowledge & goal of analysis)
df_cleaned = df_cleaned.drop({'id','Profession','City','Job Satisfaction','Work Pressure'}, axis=1)

In [10]:
df_cleaned.head()

,Gender,Age,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,Male,33.0,5.0,8.97,2.0,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,Female,24.0,2.0,5.90,5.0,5-6 hours,Moderate,BSc,No,3.0,2.0,Yes,0
2,Male,31.0,3.0,7.03,5.0,Less than 5 hours,Healthy,BA,No,9.0,1.0,Yes,0
3,Female,28.0,3.0,5.59,2.0,7-8 hours,Moderate,BCA,Yes,4.0,5.0,Yes,1
4,Female,25.0,4.0,8.13,3.0,5-6 hours,Moderate,M.Tech,Yes,1.0,1.0,No,0


In [11]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27898 entries, 0 to 27900
Data columns (total 13 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 27898 non-null  object 
 1   Age                                    27898 non-null  float64
 2   Academic Pressure                      27898 non-null  float64
 3   CGPA                                   27898 non-null  float64
 4   Study Satisfaction                     27898 non-null  float64
 5   Sleep Duration                         27898 non-null  object 
 6   Dietary Habits                         27898 non-null  object 
 7   Degree                                 27898 non-null  object 
 8   Have you ever had suicidal thoughts ?  27898 non-null  object 
 9   Work/Study Hours                       27898 non-null  float64
 10  Financial Stress                       27898 non-null  float64
 11  Family 

In [12]:
# Create label encoders for all features
label_encoders = {
    'Gender': LabelEncoder(),
    'Sleep Duration': None,  # We'll handle this one manually
    'Dietary Habits': LabelEncoder(),
    'Degree': LabelEncoder(),
    'Have you ever had suicidal thoughts ?': LabelEncoder(),
    'Family History of Mental Illness': LabelEncoder()
}

# Manually define the mapping for Sleep Duration
sleep_duration_mapping = {
    'Less than 5 hours': 0,
    '5-6 hours': 1,
    '7-8 hours': 2,
    'More than 8 hours': 3,
    'Others': 4
}

# Apply encoding for each feature
for feature, encoder in label_encoders.items():
    if feature == 'Sleep Duration':
        # Manually map the Sleep Duration
        df_cleaned[feature] = df_cleaned[feature].map(sleep_duration_mapping)
        print(f"\nMapping feature '{feature}':")
        for category, value in sleep_duration_mapping.items():
            print(f"{category} -> {value}")
    else:
        # Fit and transform using LabelEncoder
        df_cleaned[feature] = encoder.fit_transform(df_cleaned[feature])
        print(f"\nMapping feature '{feature}':")
        for category, value in zip(encoder.classes_, range(len(encoder.classes_))):
            print(f"{category} -> {value}")

print("\nDataFrame after Label Encoding:")
df_cleaned.head()


Mapping feature 'Gender':
Female -> 0
Male -> 1

Mapping feature 'Sleep Duration':
Less than 5 hours -> 0
5-6 hours -> 1
7-8 hours -> 2
More than 8 hours -> 3
Others -> 4

Mapping feature 'Dietary Habits':
Healthy -> 0
Moderate -> 1
Others -> 2
Unhealthy -> 3

Mapping feature 'Degree':
B.Arch -> 0
B.Com -> 1
B.Ed -> 2
B.Pharm -> 3
B.Tech -> 4
BA -> 5
BBA -> 6
BCA -> 7
BE -> 8
BHM -> 9
BSc -> 10
Class 12 -> 11
LLB -> 12
LLM -> 13
M.Com -> 14
M.Ed -> 15
M.Pharm -> 16
M.Tech -> 17
MA -> 18
MBA -> 19
MBBS -> 20
MCA -> 21
MD -> 22
ME -> 23
MHM -> 24
MSc -> 25
Others -> 26
PhD -> 27

Mapping feature 'Have you ever had suicidal thoughts ?':
No -> 0
Yes -> 1

Mapping feature 'Family History of Mental Illness':
No -> 0
Yes -> 1

DataFrame after Label Encoding:


,Gender,Age,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,1,33.0,5.0,8.97,2.0,1,0,3,1,3.0,1.0,0,1
1,0,24.0,2.0,5.90,5.0,1,1,10,0,3.0,2.0,1,0
2,1,31.0,3.0,7.03,5.0,0,0,5,0,9.0,1.0,1,0
3,0,28.0,3.0,5.59,2.0,2,1,7,1,4.0,5.0,1,1
4,0,25.0,4.0,8.13,3.0,1,1,17,1,1.0,1.0,0,0


# Feature Importance

In [13]:
X = df_cleaned.drop('Depression', axis=1)
y = df_cleaned['Depression']

In [14]:
y.value_counts()

Depression
1    16335
0    11563
Name: count, dtype: int64

In [15]:
random_state_ = 42
n = 10  # Number of top features to display

In [16]:
# Function to extract top features and save them
def get_top_n_features(model, model_name, n):
    if model_name == "Logistic Regression":
        importance = np.abs(model.coef_[0])
    elif model_name in ["SVM"]:
        importance = np.abs(model.coef_[0])
    else:
        importance = model.feature_importances_
    
    importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance}).sort_values(by='Importance', ascending=False)
    top_features = importance_df.head(n)
    print(f"\nTop {n} Features from {model_name}:")
    display(top_features)
    return top_features['Feature'].values

In [17]:
# 1. Random Forest Classifier
random_forest_classifier_entropy = RandomForestClassifier(criterion='entropy', random_state=random_state_)
random_forest_classifier_entropy.fit(X, y)
top_features_rf = get_top_n_features(random_forest_classifier_entropy, "Random Forest", n)


Top 10 Features from Random Forest:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,0.186472
2,Academic Pressure,0.155663
3,CGPA,0.117025
1,Age,0.100350
10,Financial Stress,0.093142
9,Work/Study Hours,0.086126
7,Degree,0.082660
4,Study Satisfaction,0.053213
5,Sleep Duration,0.044200
6,Dietary Habits,0.039663


In [18]:
# 2. Extra Trees Classifier
extra_trees_classifier = ExtraTreesClassifier(random_state=random_state_)
extra_trees_classifier.fit(X, y)
top_features_et = get_top_n_features(extra_trees_classifier, "Extra Trees", n)


Top 10 Features from Extra Trees:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,0.246000
2,Academic Pressure,0.156641
10,Financial Stress,0.098518
1,Age,0.087395
9,Work/Study Hours,0.080102
3,CGPA,0.077781
7,Degree,0.067800
4,Study Satisfaction,0.055004
5,Sleep Duration,0.045843
6,Dietary Habits,0.039933


In [19]:
# 3. Logistic Regression
logistic_model = LogisticRegression(max_iter=1000, random_state=random_state_)
logistic_model.fit(X, y)
top_features_logistic = get_top_n_features(logistic_model, "Logistic Regression", n)


Top 10 Features from Logistic Regression:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,2.505152
2,Academic Pressure,0.836814
10,Financial Stress,0.553177
6,Dietary Habits,0.353016
4,Study Satisfaction,0.244719
11,Family History of Mental Illness,0.243480
5,Sleep Duration,0.186671
9,Work/Study Hours,0.117086
1,Age,0.109285
3,CGPA,0.061648


In [20]:
# 4. AdaBoost Classifier
adaboost_classifier = AdaBoostClassifier(random_state=random_state_)
adaboost_classifier.fit(X, y)
top_features_ab = get_top_n_features(adaboost_classifier, "AdaBoost", n)


Top 10 Features from AdaBoost:


,Feature,Importance
2,Academic Pressure,0.280003
10,Financial Stress,0.163822
8,Have you ever had suicidal thoughts ?,0.146643
1,Age,0.137731
9,Work/Study Hours,0.079458
6,Dietary Habits,0.078462
4,Study Satisfaction,0.046358
5,Sleep Duration,0.032005
11,Family History of Mental Illness,0.013103
3,CGPA,0.012505


In [21]:
# 5. Support Vector Machine (SVM)
svm_model = SVC(kernel='linear', random_state=random_state_)
svm_model.fit(X, y)
top_features_svm = get_top_n_features(svm_model, "SVM", n)


Top 10 Features from SVM:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,1.777426
2,Academic Pressure,0.568597
10,Financial Stress,0.374507
6,Dietary Habits,0.226764
11,Family History of Mental Illness,0.176105
4,Study Satisfaction,0.160889
5,Sleep Duration,0.128072
9,Work/Study Hours,0.079134
1,Age,0.076812
3,CGPA,0.035219


# Modelling & Evaluation

In [22]:
# Define a function to create train-test splits for each model based on selected features, train, and evaluate the models
def train_and_evaluate_model(X, y, selected_features, model, model_name, print_result=False):
    X_selected = X[selected_features]
    X_train, X_test, y_train, y_test = train_test_split(
        X_selected, 
        y, 
        test_size=0.2, 
        random_state=random_state_, 
        stratify=y  # Add this parameter to ensure stratified sampling
    )
        
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, predictions)
    precision = precision_score(y_test, predictions, average='weighted')
    recall = recall_score(y_test, predictions, average='weighted')
    f1 = f1_score(y_test, predictions, average='weighted')
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    confusion = confusion_matrix(y_test, predictions)
    
    # Create a DataFrame for the confusion matrix with labels
    confusion_df = pd.DataFrame(confusion, 
                                 index=[f'Actual {label}' for label in np.unique(y)], 
                                 columns=[f'Predicted {label}' for label in np.unique(y)])
    
    if print_result:
        print(f"\n{model_name} Evaluation Metrics:")
        print(f"Accuracy: {accuracy:.2f}")
        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1 Score: {f1:.2f}")
        print(f"ROC AUC: {roc_auc:.2f}")
        print("Confusion Matrix:\n", confusion_df)
    
    return {
        "model_name": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "confusion_df": confusion_df
    }

In [23]:
models = {
    "Random Forest": (RandomForestClassifier(criterion='entropy', random_state=random_state_), top_features_rf),
    'Extra Trees': (ExtraTreesClassifier(random_state=random_state_), top_features_et),
    "Logistic Regression": (LogisticRegression(max_iter=1000, random_state=random_state_), top_features_logistic),
    "AdaBoost": (AdaBoostClassifier(random_state=random_state_), top_features_ab),
    "SVM": (SVC(kernel='linear', probability=True, random_state=random_state_), top_features_svm),
}

In [24]:
evaluations = []

# Wrap models.items() with tqdm for a progress bar
for model_name, (model, selected_features) in tqdm(models.items(), desc="Training & Evaluating Models", unit="model"):
    eval_metrics = train_and_evaluate_model(X, y, selected_features, model, model_name)
    evaluations.append(eval_metrics)

Training & Evaluating Models: 100%|██████████| 5/5 [02:23<00:00, 28.66s/model]


In [25]:
eval_df = pd.DataFrame(evaluations)
eval_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.832437,0.831797,0.832437,0.831790,0.907609
1,Extra Trees,0.829928,0.829273,0.829928,0.829298,0.905043
2,Logistic Regression,0.846057,0.845536,0.846057,0.845511,0.918824
3,AdaBoost,0.844624,0.844126,0.844624,0.843848,0.918445
4,SVM,0.845699,0.845173,0.845699,0.845127,0.918761


In [26]:
# Create an empty list to store confusion matrix DataFrames
confusion_matrices_dfs = []

for i in range(len(eval_df)):
    model_name = eval_df['model_name'].iloc[i]  # Get the model name
    confusion_matrix_ = eval_df['confusion_df'].iloc[i]  # Get the confusion matrix
    
    # Convert each confusion matrix to a DataFrame
    labels = np.unique(y)  # Get the unique labels
    confusion_df = pd.DataFrame(confusion_matrix_, 
                                 index=[f'Actual {label}' for label in labels[::-1]],  # Reverse order for Actual
                                 columns=[f'Predicted {label}' for label in labels[::-1]])  # Reverse order for Predicted
    confusion_matrices_dfs.append(confusion_df)
    
    # Print each confusion matrix DataFrame
    print(f"\nConfusion Matrix for {model_name}:")
    print(confusion_df)


Confusion Matrix for Random Forest:
          Predicted 1  Predicted 0
Actual 1         2854          413
Actual 0          522         1791

Confusion Matrix for Extra Trees:
          Predicted 1  Predicted 0
Actual 1         2845          422
Actual 0          527         1786

Confusion Matrix for Logistic Regression:
          Predicted 1  Predicted 0
Actual 1         2888          379
Actual 0          480         1833

Confusion Matrix for AdaBoost:
          Predicted 1  Predicted 0
Actual 1         2902          365
Actual 0          502         1811

Confusion Matrix for SVM:
          Predicted 1  Predicted 0
Actual 1         2889          378
Actual 0          483         1830


# Feature Engineering

In [27]:
for model_name, (model, selected_features) in models.items():
    print(f"Selected Features for {model_name}:")
    display(selected_features)

Selected Features for Random Forest:


array(['Have you ever had suicidal thoughts ?', 'Academic Pressure',
       'CGPA', 'Age', 'Financial Stress', 'Work/Study Hours', 'Degree',
       'Study Satisfaction', 'Sleep Duration', 'Dietary Habits'],
      dtype=object)

Selected Features for Extra Trees:


array(['Have you ever had suicidal thoughts ?', 'Academic Pressure',
       'Financial Stress', 'Age', 'Work/Study Hours', 'CGPA', 'Degree',
       'Study Satisfaction', 'Sleep Duration', 'Dietary Habits'],
      dtype=object)

Selected Features for Logistic Regression:


array(['Have you ever had suicidal thoughts ?', 'Academic Pressure',
       'Financial Stress', 'Dietary Habits', 'Study Satisfaction',
       'Family History of Mental Illness', 'Sleep Duration',
       'Work/Study Hours', 'Age', 'CGPA'], dtype=object)

Selected Features for AdaBoost:


array(['Academic Pressure', 'Financial Stress',
       'Have you ever had suicidal thoughts ?', 'Age', 'Work/Study Hours',
       'Dietary Habits', 'Study Satisfaction', 'Sleep Duration',
       'Family History of Mental Illness', 'CGPA'], dtype=object)

Selected Features for SVM:


array(['Have you ever had suicidal thoughts ?', 'Academic Pressure',
       'Financial Stress', 'Dietary Habits',
       'Family History of Mental Illness', 'Study Satisfaction',
       'Sleep Duration', 'Work/Study Hours', 'Age', 'CGPA'], dtype=object)

In [28]:
# Normalization with MinMaxScaler

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Specify the features to normalize (you can adjust this list)
features_to_normalize = df_cleaned.columns

# Fit the scaler on the specified features and transform them
df_normalized = df_cleaned.copy()
df_normalized[features_to_normalize] = scaler.fit_transform(df_normalized[features_to_normalize])

# Display the DataFrame after normalization
print("\nDataFrame after Min-Max Normalization:")
df_normalized.head()


DataFrame after Min-Max Normalization:


,Gender,Age,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,1.0,0.365854,1.0,0.897,0.4,0.25,0.000000,0.111111,1.0,0.250000,0.00,0.0,1.0
1,0.0,0.146341,0.4,0.590,1.0,0.25,0.333333,0.370370,0.0,0.250000,0.25,1.0,0.0
2,1.0,0.317073,0.6,0.703,1.0,0.00,0.000000,0.185185,0.0,0.750000,0.00,1.0,0.0
3,0.0,0.243902,0.6,0.559,0.4,0.50,0.333333,0.259259,1.0,0.333333,1.00,1.0,1.0
4,0.0,0.170732,0.8,0.813,0.6,0.25,0.333333,0.629630,1.0,0.083333,0.00,0.0,0.0


In [29]:
df_normalized.describe()

,Gender,Age,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
count,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000,27898.000000
mean,0.557244,0.190785,0.628267,0.765616,0.588795,0.350177,0.488601,0.404292,0.632877,0.596381,0.534967,0.483942,0.585526
std,0.496721,0.119650,0.276292,0.147071,0.272224,0.282071,0.413225,0.267990,0.482029,0.308967,0.359337,0.499751,0.492640
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.073171,0.400000,0.629000,0.400000,0.000000,0.000000,0.185185,0.000000,0.333333,0.250000,0.000000,0.000000
50%,1.000000,0.170732,0.600000,0.777000,0.600000,0.250000,0.333333,0.407407,1.000000,0.666667,0.500000,0.000000,1.000000
75%,1.000000,0.292683,0.800000,0.892000,0.800000,0.500000,1.000000,0.592593,1.000000,0.833333,0.750000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Re-Modelling & Evaluation

In [30]:
X_norm = df_normalized.drop('Depression', axis=1)
y = df_normalized['Depression']

In [31]:
re_evaluations = []

# Wrap models.items() with tqdm for a progress bar
for model_name, (model, selected_features) in tqdm(models.items(), desc="Training Models", unit="model"):
    eval_metrics = train_and_evaluate_model(X_norm, y, selected_features, model, model_name)
    re_evaluations.append(eval_metrics)

Training Models: 100%|██████████| 5/5 [01:30<00:00, 18.19s/model]


In [32]:
re_eval_df = pd.DataFrame(re_evaluations)
re_eval_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.832258,0.831618,0.832258,0.831617,0.907754
1,Extra Trees,0.829928,0.829273,0.829928,0.829298,0.905043
2,Logistic Regression,0.845878,0.845355,0.845878,0.845325,0.918844
3,AdaBoost,0.844624,0.844126,0.844624,0.843848,0.918445
4,SVM,0.845699,0.845173,0.845699,0.845127,0.918754


In [33]:
eval_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.832437,0.831797,0.832437,0.831790,0.907609
1,Extra Trees,0.829928,0.829273,0.829928,0.829298,0.905043
2,Logistic Regression,0.846057,0.845536,0.846057,0.845511,0.918824
3,AdaBoost,0.844624,0.844126,0.844624,0.843848,0.918445
4,SVM,0.845699,0.845173,0.845699,0.845127,0.918761


# Feature Importance with Threshold

In [34]:
treshold = 0.05

In [35]:
def get_top_features(model, model_name, threshold):
    """
    Extracts features with importance greater than the specified threshold.

    Parameters:
    - model: Trained model (e.g., Logistic Regression, SVM, Random Forest, etc.).
    - model_name: Name of the model (e.g., "Logistic Regression", "SVM", "Random Forest").
    - threshold: Importance threshold for selecting features.

    Returns:
    - top_features: Array of feature names with importance greater than the threshold.
    """
    # Extract feature importance based on the model type
    if model_name == "Logistic Regression":
        importance = np.abs(model.coef_[0])  # Absolute coefficients for Logistic Regression
    elif model_name in ["SVM"]:
        importance = np.abs(model.coef_[0])  # Absolute coefficients for SVM
    else:
        importance = model.feature_importances_  # Feature importances for tree-based models

    # Create a DataFrame of feature importances
    importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': importance})
    
    # Filter features based on the threshold
    top_features = importance_df[importance_df['Importance'] > threshold]
    
    # Display the results
    print(f"\nFeatures from {model_name} with Importance > {threshold}:")
    display(top_features.sort_values(by='Importance', ascending=False))
    
    return top_features['Feature'].values

In [36]:
# 1. Random Forest Classifier
top_features_v2_rf = get_top_features(random_forest_classifier_entropy, "Random Forest", treshold)


Features from Random Forest with Importance > 0.05:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,0.186472
2,Academic Pressure,0.155663
3,CGPA,0.117025
1,Age,0.100350
10,Financial Stress,0.093142
9,Work/Study Hours,0.086126
7,Degree,0.082660
4,Study Satisfaction,0.053213


In [37]:
# 2. Extra Trees Classifier
top_features_v2_et = get_top_features(extra_trees_classifier, "Extra Trees", treshold)


Features from Extra Trees with Importance > 0.05:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,0.246000
2,Academic Pressure,0.156641
10,Financial Stress,0.098518
1,Age,0.087395
9,Work/Study Hours,0.080102
3,CGPA,0.077781
7,Degree,0.067800
4,Study Satisfaction,0.055004


In [38]:
# 3. Logistic Regression
top_features_v2_logistic = get_top_features(logistic_model, "Logistic Regression", treshold)


Features from Logistic Regression with Importance > 0.05:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,2.505152
2,Academic Pressure,0.836814
10,Financial Stress,0.553177
6,Dietary Habits,0.353016
4,Study Satisfaction,0.244719
11,Family History of Mental Illness,0.243480
5,Sleep Duration,0.186671
9,Work/Study Hours,0.117086
1,Age,0.109285
3,CGPA,0.061648


In [39]:
# 4. AdaBoost Classifier
top_features_v2_ab = get_top_features(adaboost_classifier, "AdaBoost", treshold)


Features from AdaBoost with Importance > 0.05:


,Feature,Importance
2,Academic Pressure,0.280003
10,Financial Stress,0.163822
8,Have you ever had suicidal thoughts ?,0.146643
1,Age,0.137731
9,Work/Study Hours,0.079458
6,Dietary Habits,0.078462


In [40]:
# 5. Support Vector Machine (SVM)
top_features_v2_svm = get_top_features(svm_model, "SVM", treshold)


Features from SVM with Importance > 0.05:


,Feature,Importance
8,Have you ever had suicidal thoughts ?,1.777426
2,Academic Pressure,0.568597
10,Financial Stress,0.374507
6,Dietary Habits,0.226764
11,Family History of Mental Illness,0.176105
4,Study Satisfaction,0.160889
5,Sleep Duration,0.128072
9,Work/Study Hours,0.079134
1,Age,0.076812


# Re-Modelling & Evaluation (New Selected Features)

In [41]:
models_v2 = {
    "Random Forest": (RandomForestClassifier(criterion='entropy', random_state=random_state_), top_features_v2_rf),
    'Extra Trees': (ExtraTreesClassifier(random_state=random_state_), top_features_v2_et),
    "Logistic Regression": (LogisticRegression(max_iter=1000, random_state=random_state_), top_features_v2_logistic),
    "AdaBoost": (AdaBoostClassifier(random_state=random_state_), top_features_v2_ab),
    "SVM": (SVC(kernel='linear', probability=True, random_state=random_state_), top_features_v2_svm),
}

In [42]:
re_evaluations_v2 = []

# Wrap models.items() with tqdm for a progress bar
for model_name, (model, selected_features) in tqdm(models_v2.items(), desc="Training Models", unit="model"):
    eval_metrics = train_and_evaluate_model(X_norm, y, selected_features, model, model_name)
    re_evaluations_v2.append(eval_metrics)

Training Models: 100%|██████████| 5/5 [01:29<00:00, 17.94s/model]


In [43]:
re_eval_v2_df = pd.DataFrame(re_evaluations_v2)
re_eval_v2_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]  # Normalization, top features with 0.05 treshold importance

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.825986,0.825302,0.825986,0.825355,0.902854
1,Extra Trees,0.822222,0.821470,0.822222,0.821372,0.897859
2,Logistic Regression,0.845878,0.845355,0.845878,0.845325,0.918844
3,AdaBoost,0.836022,0.835480,0.836022,0.835065,0.912841
4,SVM,0.847133,0.846619,0.847133,0.846590,0.918434


In [44]:
re_eval_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]  # Normalization, top 10 features

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.832258,0.831618,0.832258,0.831617,0.907754
1,Extra Trees,0.829928,0.829273,0.829928,0.829298,0.905043
2,Logistic Regression,0.845878,0.845355,0.845878,0.845325,0.918844
3,AdaBoost,0.844624,0.844126,0.844624,0.843848,0.918445
4,SVM,0.845699,0.845173,0.845699,0.845127,0.918754


In [45]:
eval_df[['model_name', 'accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]  # No Normalization, top 10 features

,model_name,accuracy,precision,recall,f1_score,roc_auc
0,Random Forest,0.832437,0.831797,0.832437,0.831790,0.907609
1,Extra Trees,0.829928,0.829273,0.829928,0.829298,0.905043
2,Logistic Regression,0.846057,0.845536,0.846057,0.845511,0.918824
3,AdaBoost,0.844624,0.844126,0.844624,0.843848,0.918445
4,SVM,0.845699,0.845173,0.845699,0.845127,0.918761


# Best Features for Each Model
1. Random Forest: no normalization, top 10 features
2. Extra Trees: no normalization, top 10 features
3. Logistiic Regression: no normalization, top 10 features -> Best Performance
4. AdaBoost: no normalization, top 10 features
5. SVM: normalization, top features with 0.05 threshold imporatance

# Hyperparameter Tuning

In [46]:
# Logistic Regression
from sklearn.model_selection import ParameterGrid

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X[top_features_logistic], y, test_size=0.2, random_state=random_state_, stratify=y
)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'penalty': ['l1', 'l2', 'elasticnet', None],  # Regularization type
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # Optimization algorithm
    'max_iter': [100, 200, 500],  # Maximum number of iterations
    'class_weight': [None, 'balanced']  # Weights associated with classes
}

# Initialize the Logistic Regression model
lr_model = LogisticRegression(random_state=random_state_)

# Track the best accuracy and corresponding hyperparameters
best_accuracy = 0
best_hyperparams = None

# Custom callback function to log the best accuracy so far
def log_top_accuracy(results):
    global best_accuracy, best_hyperparams
    current_accuracy = results['mean_test_score'].max()
    if current_accuracy > best_accuracy:
        best_accuracy = current_accuracy
        best_hyperparams = results['params'][results['mean_test_score'].argmax()]
        print(f"\nNew Best Accuracy: {best_accuracy:.4f}")
        print(f"Hyperparameters: {best_hyperparams}")

# Set up GridSearchCV with verbose logging
grid_search = GridSearchCV(
    estimator=lr_model,
    param_grid=param_grid,
    scoring='accuracy',  # Metric to evaluate (can be changed to 'f1', 'roc_auc', etc.)
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available CPU cores
    verbose=2,  # Print logs for each iteration
    return_train_score=True  # Return training scores for logging
)

# Fit the GridSearchCV and log results for every iteration
print("\nGrid Search Progress:")

for i in range(len(ParameterGrid(param_grid))):
    grid_search.fit(X_train, y_train)
    
    # Get all results from this iteration
    results = grid_search.cv_results_
    
    # Log each iteration's best accuracy
    current_accuracies = results['mean_test_score']
    current_params = results['params']
    
    for acc, params in zip(current_accuracies, current_params):
        print(f"Iteration {i+1}: Accuracy = {acc * 100:.2f}% | Hyperparameters: {params}")
    
    # Check and update the best accuracy
    log_top_accuracy(results)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"\nBest Parameters: {best_params}")
print(f"Best Cross-Validation Accuracy: {best_score * 100:.2f}%")

# Evaluate the best model on the test set
best_lr_model = grid_search.best_estimator_
y_pred = best_lr_model.predict(X_test)

print("\nTest Set Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Grid Search Progress:
Fitting 5 folds for each of 720 candidates, totalling 3600 fits


KeyboardInterrupt: 

- Grid Search taking too long and not significantly got any better performance, let's just go with the original model

# Save Model

In [ ]:
X = df_cleaned.drop('Depression', axis=1)
y = df_cleaned['Depression']

X_selected = X[selected_features]
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, 
    y, 
    test_size=0.2, 
    random_state=random_state_, 
    stratify=y  # Add this parameter to ensure stratified sampling
)

# Train the Random Forest model
best_model = LogisticRegression(max_iter=1000, random_state=random_state_)
best_model.fit(X_train, y_train)

# Make predictions
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Set Accuracy: {accuracy * 100:.2f}%\n")
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Save the trained model to a file
with open("logistic_regression_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("\nModel saved successfully as 'logistic_regression_model.pkl'!")

Test Set Accuracy: 84.62%

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.79      0.81      2313
           1       0.86      0.88      0.87      3267

    accuracy                           0.85      5580
   macro avg       0.84      0.84      0.84      5580
weighted avg       0.85      0.85      0.85      5580


Model saved successfully as 'logistic_regression_model.pkl'!


# Inference Test

In [48]:
top_features_logistic

array(['Have you ever had suicidal thoughts ?', 'Academic Pressure',
       'Financial Stress', 'Dietary Habits', 'Study Satisfaction',
       'Family History of Mental Illness', 'Sleep Duration',
       'Work/Study Hours', 'Age', 'CGPA'], dtype=object)

In [49]:
def predict_depression(new_data):
    # Load the model
    with open("logistic_regression_model.pkl", "rb") as f:
        model = pickle.load(f)
    
    # Preprocess the input (assuming new_data is already encoded)
    prediction = model.predict(new_data)
    prediction_proba = model.predict_proba(new_data)
    
    return {
        'prediction': prediction[0],
        'probability': prediction_proba[0].max()
    }

# Example usage
new_sample = X_selected.iloc[[0]]  # Use first row from training data as example
result = predict_depression(new_sample)
print(f"Depression Prediction: {result['prediction']}")
print(f"Confidence: {result['probability'] * 100:.2f}%")

Depression Prediction: 1
Confidence: 55.77%


In [50]:
# Create a new sample row for prediction
new_sample_data = pd.DataFrame({
    'Have you ever had suicidal thoughts ?': [1],  # Encoded value
    'Academic Pressure': [8],
    'Financial Stress': [7],
    'Dietary Habits': [1],  # Encoded value
    'Study Satisfaction': [2],
    'Family History of Mental Illness': [1],  # Encoded value
    'Sleep Duration': [0],  # Less than 5 hours
    'Work/Study Hours': [12],
    'Age': [21],
    'CGPA': [3.2]
})

# Ensure the new data uses the same columns as training data
new_sample = new_sample_data[selected_features]

# Make prediction
result = predict_depression(new_sample)
print(f"Depression Prediction: {result['prediction']}")
print(f"Confidence: {result['probability'] * 100:.2f}%")

Depression Prediction: 1
Confidence: 99.99%
